# Netcdf C++ API crashes on Theia

## Notes, 7/29/19

* Dan Holdaway was testing on Theia and experienced test failures that turned out to be the program crashing

* I was able to reproduce the crashes on Cheyenne
    * Intel 17.0.1 compilers
    * ioda_fileio test
        * Message: NetCDF: Not a valid ID, file: ../../cxx4/ncFile.cpp line:33
            * This turned out to be a leftover call to nc_close() from the C API
            * Removing this call made the error go away, but did not fix the crash problem
        * Discovered that the run dies in the ReadVar_helper() routine
    * The crashes go away if I run in Debug mode
    * The tests pass on AWS with Intel 17.0.1
        * Also with new eckit (1.1.0) and fckit (0.6.3)

## Notes, 8/1/19

* Tried different optimization levels on Cheyenne
   * -O0: Debug mode, no crashes for both ioda and ufo
       * other test failures
   * -O1: no crahses for both ioda and ufo
   * -O2: crashes for both ioda and ufo

* Ran DDT
   * Discovered the crash occurs during the destructor of a std::string
       * An invalid pointer was being passed to the free function
   * This was fixed by initializing the NcVarName string in ReadVar_helper with " " (one blank space).
       * Should not have to do this
   * Another crash surfaced
       * During failIfUnset() in the DateTime class
       * ObsSpace constructor
       * InitFromFile() routine
       * if (test_dt_ > winbgn_) && (test_dt_ <= winend_)
           * crash is during the second expression: (test_dt_ <+ winend_)


# Bump test failures on Mac

## Notes, 7/31/19

* Tests pass on Yannick's iMac

* Same tests fail on my iMac and MacBook
    * 18 bump tests
    * run to completion, but results are out of tolerance

* My iMac and MacBook:
    * High Sierra (10.13.6)
    * Clang 10.0.0
    * GNU 7.4.0
    * openMPI 3.1.2

* Yannick's iMac
    * Mohave (10.14.6)
    * Clang 10.0.1
    * GNU 9.1.0
    * Mpich2 3.3.1

* Updated my iMac to match Yannick's but the tests continued to fail

* Yannick does not use mpi wrappers (compiles directly with clang, clang++ and gfortran)
    * When I try that on my iMac, the 18 tests continue to fail

## Notes, 8/1/19

* Checked Yannick's machine for more differences
    * netcdf 4.6.3
    * boost 1.69.0
    * eigen 3.3.7
    * cmake 3.14.5
    * mpich 3.3.1
    
    * discovered that Yannick's machine was missing nccmp and the compare script in bump was erroneously returning zero in this case
        * Installed nccmp on Yannick's iMac, and now the bump tests fail

* More experiments from my iMac
    * Mojave, Clang 10.0.1, GNU 9.1.0
    * openMPI 3.1.2
        * crashes on many bump tests
        * seg fault, backtrace doesn't reveal much
        * same result using mpi wrappers, and compiling without mpi wrappers
    * mpich2 3.3.1
        * 18 bump errors
        * with mpi wrappers, one additional failure
            * intset_parser test in oops
        * without mpi wrappers
            * intset_parser test passes

## Notes, 8/2/19

* Mark O was able to get the tests to work on machines that experienced the failures by making sure the compile was using the low-level "generic" architecture
  * compiler options that look like: -m arch=generic
  * Mark also compiled without the mpi wrappers